In [ ]:
import lammps

from pathlib import Path
import pandas as pd

## Defining constants

In [ ]:
ENERGY_EVAL_INP = '''
units real
atom_style full

dimension 3
boundary p p p

pair_style lj/cut/coul/cut 19.99999 19.99999
pair_modify mix geometric

bond_style hybrid harmonic
angle_style hybrid harmonic
dihedral_style hybrid fourier
improper_style cvff
special_bonds lj 0.0 0.0 0.5 coul 0.0 0.0 0.5

read_data $INP_FILE
thermo_style yaml ebond eangle edihed eimp epair evdwl ecoul elong etail pe ke etotal
run 0
'''

E_ALL = {
    'Bond'              : 'ebond',
    'Angle'             : 'eangle',
    'Dihedral'          : 'edihed',
    'Impropers'         : 'eimp',
    'vdW pairwise'      : 'evdwl',
    'Pairwise'          : 'epair',
    'Coulomb'           : 'ecoul',
    'Ranged k-space'    : 'elong',
    'vdW ranged (kcal)' : 'etail',
    'Potential'         : 'pe',
    'Kinetic'           : 'ke',
    'Total'             : 'etotal'
}

E_TO_REPORT = {
    'Nonbonded (kcal/mol)' : 'epair',
    'Bond (kcal/mol)'      : 'ebond',
    'Angle (kcal/mol)'     : 'eangle',
    'Torsion (kcal/mol)'   : 'edihed',
    'Impropers (kcal/mol)' : 'eimp',
    'Potential (kcal/mol)' : 'pe',
    'Kinetic (kcal/mol)'   : 'ke',
    # 'Total (kcal/mol)'     : 'etotal'
}

In [ ]:
lammps_dir = Path('LAMMPS')

records = {}
for subdir in lammps_dir.iterdir():
    if subdir.is_dir():
        chemistry = subdir.name
        for lammps_file in subdir.iterdir():
            mol_name = lammps_file.stem
            with lammps.lammps() as lmp: # need to create new lammps() object instance for each run
                lmp.commands_string( ENERGY_EVAL_INP.replace('$INP_FILE', str(lammps_file)) )
                rep_energies = {
                    label : lmp.get_thermo(prop_tag)
                        for label, prop_tag in E_TO_REPORT.items()
                }

            records[(chemistry, mol_name)] = rep_energies

In [ ]:
df = pd.DataFrame.from_dict(records, 'index')
df.index.names  = ['Chemistry', 'Molecule'] # ensure index labels are labelled consistently
df.to_csv(lammps_dir/'LAMMPS_PEs.csv')